### 구글드라이브 연결
> - drive 폴더가 붙은 것을 확인한다.
> - drwx------ 4 root root 4.0K Mar 22 11:21 drive

In [1]:
# from google.colab import drive 
# drive.mount('/content/drive/')

### Install / Import 모듈 

In [2]:
# 코랩에 없는 라이브러리 설치해주기
# !pip install bayesian-optimization

In [3]:
import os
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지

from tqdm import tqdm                       # 진행바

from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings           
import random                       
warnings.filterwarnings("ignore")           # 경고 문구 미표시

In [4]:
# 데이터가 많아 5분 정도 걸립니다.  
# 전체경로 = /content/drive/My Drive/Colab Notebooks/competition/c03_starcraft_prediction/data_raw/train.csv
# /content/drive/My Drive/Colab Notebooks/
# dir_base = '/content/drive/My Drive/Colab Notebooks/competition/''

dir_base = '/home/yk/0325_Starcraft/competition/'

raw      = 'c03_starcraft_prediction/data_raw/'
remake   = 'c03_starcraft_prediction/data_remake/'
submit   = 'c03_starcraft_prediction/data_submit/'
                                                           # WAY-01 = LGBM
filename_submit = 'submission_way02_15_75_0412.csv'        # WAY-02 = XGBM

### 사용함수의 정의

In [5]:
os.chdir(dir_base)
from _assets.modules import *
from _assets.module_data_preps import *
from _assets.module_xgbm_model import *


# OS 화일 및 DF 정보조회를 위한 탐색 모듈


# GAME_ID 유니크 데이터 전처리를 위한 모듈 - baseline offer


# XGBM_CV 모델 - by SongDo_StudyGroup Code 참조



Using TensorFlow backend.


### Data 불러오기

| 컬럼명  | 값    | type     | 설명                  | 비고   |
|:-------:|:------|----------|-----------------------|--------|
| game_id |33781  |int       |경기 구분 기호         | unique (multiple) 
| winner  |0.6523 |float     |player-1 승리확률      | 0.6287
| time    |2.24   |float     |경기시간 (60분법 구분) | 2분24초
| player  |0      |int       |player 0 - 첫번째 선수 
|         |1      |int       |player 1 - 두번째 선수          
| species |T      |Object    |테란 (Terran)    |     
|         |P      |Object    |프로토스 (Protos)|      
|         |Z      |Object    |저그 (Zerg)      |      
| event   |Ability           |Object |생산, 공격 등 선수의 주요 행동
|         |AddToControlGroup |Object |부대에 추가
|         |Camera            |Object |시점 선택
|         |ControlGroup      |Object |부대 행동
|         |GetControlGroup   |Object |부대 불러오기
|         |Right             |Object |Click : 마우스 우클릭
|         |Selection         |Object |객체 선택
|         |SetControlGroup   |Object |부대 지정
|event_contents|@(좌표, 등)  | object|이벤트에 관한 상세설명 

In [6]:
# 화일위치 / list 확인하기 ... 사용함수
# show_ls('/content')          # drive 가 붙었는지 확인!
show_ls(dir_base + raw)      # 불러올 Data 화일이름 확인!

False
DIR_TARGET=/home/yk/0325_Starcraft/competition
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. sample_submission.csv
  02. test.csv
  03. train.csv


In [7]:
%%time
# raw 데이터 3개 불러오기 ... 2분13초

train = pd.read_csv(dir_base + raw + "train.csv")                           # [ 67,091,776 x 7 ] ... 70%
test = pd.read_csv(dir_base + raw + "test.csv")                             # [ 28,714,849 x 6 ] ... 30%
sample_submission = pd.read_csv(dir_base + raw + "sample_submission.csv")   # [     16,787 x 2 ] ... 0.0175 %

CPU times: user 1min, sys: 4.77 s, total: 1min 4s
Wall time: 1min


### Data 전처리 및 저장하기
- 전처리 된 x_train, y_train 내용을 확인하고, 저장한다.

In [8]:
show_ls(dir_base + remake)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake/
----------------------------------------
  01. .ipynb_checkpoints
  02. accumujated_winning_rate_per_time.csv
  03. accumulated_winning_rate_per_time.csv
  04. df_new_rate04_correct.csv
  05. df_win_rate_every_10sec.csv
  06. df_win_rate_every_20sec.csv
  07. df_win_rate_every_40sec.csv
  08. df_xtest_remake.csv
  09. df_xtrain_remake.csv
  10. df_xtrain_remake_final.csv
  11. df_ytrain_remake.csv


In [9]:
%%time
"""
# 데이터 전처리 과정 - 약 7 ~ 10분 정도 걸립니다 ... 여기서 세션 리셋 됨!
# x_train, y_train = data_preparation(train, answer=True)           # [ 67,091,776 x 7 ] ... 70%
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 27 ] ... [ game_id ... ]
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ] .. ['Unnamed: 0', 'game_id', 'winner']
"""

x_train = pd.read_csv(dir_base + remake + 'df_xtrain_remake.csv')  # to <class 'pandas.core.frame.DataFrame'>
ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  # to <class 'numpy.ndarray'>
y_train = ytrain['winner'].values                                 # array([1, 1, 0, ..., 0, 1, 0])

CPU times: user 54 ms, sys: 4.16 ms, total: 58.2 ms
Wall time: 57.6 ms


In [10]:
pd.Series(y_train).value_counts()
# 1    19499
# 0    19373            total = 38,872  y_train = np.array 38,872

1    19499
0    19373
dtype: int64

In [11]:
# 제대로 작성이 됬는지 확인 다 끝났으면, 저장한다. 저장끝났으면 비활성화 처리


"""  # 저장했으면 비활성 처리
df_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')
df_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')
"""

"  # 저장했으면 비활성 처리\ndf_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')\ndf_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')\n"

# XGBM Classifier 모델 적용
 - 송도 스터디그룹에서 유창준님 - XGBM 모델 코드 공유받음!

In [12]:
%%time
# 모델 그 외 변수는 고정
var_fixed = partial(
        XGB_cv, 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='score'
    )

# 베이지안 최적화 범위 설정
XGBo = BayesianOptimization(
        var_fixed, 
        {
            'max_depth': (8, 512),          # ~ 500
            'learning_rate': (0.1, 0.6),    # 0.6xxx
            'subsample': (0.5, 0.9),        # 0.5 ~ 0.7
            'colsample_bytree': (0.5, 1),   # 0.5 ~ 0.8
            # 'colsample_bynode':(0, 1),
            # 'n_estimators' : (16, 1024),
            'reg_alpha' : (7, 20),          # L1 - 7 ~ 20
            'reg_lambda' : (8, 50),         # L2 - 8 ~ 45
            'max_delta_step' : (1, 25),     # 8.x
            'gamma' : (5, 20),              # 1 ~ 15
        },
        random_state = random.randrange(50000) # 유동 시드 1~50000
    )

# XGBo.maximize(init_points=15, n_iter=60)  # 처음 15회 랜덤값으로 score 계산 후 60회 최적화 = 4/04(토)
XGBo.maximize(init_points=15, n_iter=75)    # 처음 15회 랜덤값으로 score 계산 후 75회 최적화 = 4/11(토)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.639    |  0.7925   |  16.57    |  0.1147   |  15.1     |  66.91    |  14.66    |  8.107    |  0.5789   |
|  2        |  0.6475   |  0.5554   |  6.378    |  0.139    |  17.59    |  49.96    |  10.58    |  29.84    |  0.7913   |
|  3        |  0.6426   |  0.635    |  12.28    |  0.1061   |  3.524    |  291.9    |  15.44    |  9.488    |  0.861    |
|  4        |  0.6389   |  0.5168   |  15.43    |  0.2837   |  22.26    |  130.5    |  16.63    |  39.79    |  0.7117   |
|  5        |  0.6389   |  0.5605   |  16.19    |  0.16     |  3.254    |  386.7    |  12.54    |  41.7     |  0.678    |
|  6        |  0.6397   |  0.7077   |  6.655    |  0.543    |  14.18    |  172.7    |  9.892    |  41.13    |  0.8383   |
|  7        |  0.6384   

|  41       |  0.6454   |  0.7911   |  5.295    |  0.1943   |  5.412    |  102.0    |  19.86    |  15.78    |  0.6004   |
|  42       |  0.6434   |  0.8703   |  7.164    |  0.3356   |  1.792    |  248.7    |  18.43    |  9.566    |  0.677    |
|  43       |  0.6459   |  0.5623   |  6.605    |  0.1699   |  23.26    |  34.64    |  8.387    |  49.41    |  0.6882   |
|  44       |  0.6364   |  0.8769   |  17.42    |  0.1485   |  1.534    |  375.8    |  18.94    |  9.144    |  0.5044   |
|  45       |  0.6331   |  0.9756   |  5.547    |  0.5017   |  4.506    |  402.4    |  7.736    |  26.56    |  0.7957   |
|  46       |  0.6376   |  0.5792   |  18.46    |  0.1948   |  6.779    |  495.4    |  19.4     |  45.18    |  0.8176   |
|  47       |  0.6401   |  0.6779   |  14.78    |  0.1032   |  24.91    |  439.9    |  18.08    |  10.75    |  0.847    |
|  48       |  0.6458   |  0.9545   |  5.602    |  0.2767   |  22.87    |  378.5    |  18.06    |  49.67    |  0.881    |
|  49       |  0.6393   

|  82       |  0.6464   |  0.9651   |  5.092    |  0.1338   |  9.778    |  477.3    |  19.87    |  11.68    |  0.8471   |
|  83       |  0.6381   |  0.817    |  19.0     |  0.144    |  24.14    |  38.49    |  19.05    |  47.22    |  0.8911   |
|  84       |  0.648    |  0.9422   |  6.85     |  0.1128   |  3.249    |  212.5    |  8.206    |  8.957    |  0.7894   |
|  85       |  0.6481   |  0.9264   |  5.008    |  0.1009   |  7.544    |  236.6    |  16.75    |  18.11    |  0.8075   |
|  86       |  0.648    |  0.9341   |  6.196    |  0.1443   |  24.48    |  54.32    |  12.71    |  15.24    |  0.8455   |
|  87       |  0.6469   |  0.76     |  5.457    |  0.159    |  14.42    |  510.0    |  10.06    |  49.35    |  0.8416   |
|  88       |  0.6476   |  0.8503   |  5.288    |  0.1088   |  24.52    |  210.2    |  7.981    |  21.49    |  0.837    |
|  89       |  0.6466   |  0.5432   |  5.102    |  0.1098   |  24.51    |  81.93    |  7.117    |  11.96    |  0.6966   |
|  90       |  0.6459   

### 3.0 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

In [13]:
"""
# params = lgbBO.max['params']
# models = lgb_cv(
#         params['num_leaves'], 
#         params['learning_rate'], 
#         params['n_estimators'], 
#         params['subsample'], 
#         params['colsample_bytree'], 
#         params['reg_alpha'], 
#         params['reg_lambda'], 
#         x_data=x_train, 
#         y_data=y_train, 
#         n_splits=5, 
#         output='model',
#    )
"""

params = XGBo.max['params']
models = XGB_cv(
        params['max_depth'], 
        params['learning_rate'],
        params['subsample'], 
        params['colsample_bytree'],
        # params['colsample_bynode'], 
        params['reg_alpha'], 
        params['reg_lambda'],
        params['max_delta_step'],
        # params['n_estimators'],
        params['gamma'],
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model'
    )

# TEST를 풀기위해, 학습데이터로 변경 (data_preps)
 1. Train Data set = [ 67,091,776 x 7 ] ... 70%
  - game_id 별로 정리하면 [38,872 x 27]로 압축된다. 
  - 경기당, 평균 1,726개의 이벤트가 존재한다.(압축률 0.057939 %)
 > - x_train set = *** DATA SHAPE = [ 38,872 x 27 ] pandas.DataFrame
 > - y_train set = *** DATA SHAPE = [ 38,872 x 1 ]  np.array
 
 1. Test Data set  = [ 28,714,849 x 6 ] ... 30%
  - game_id 별로 정리하면 [16,787 x 27] 로 압축된다. 
  - 경기당 평균 1711개의 이벤트가 존재한다.(압축률 0.058461 %)
 > - x_test set = *** DATA SHAPE = [16,787 x 27] pandas.DataFrame 
 > - y_test set = *** DATA SHAPE = [16,787 x 1] np.array =  대회주관자가 가지고 있음
 
 1. Submission = y_test set 
  - submission = [16,787 x 1] np.array to [16,787 x 2 ] pandas.DataFrame --> df.csv

In [14]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]


# 예측결과 Submission 화일 만들기

In [15]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [16]:
show_ls(dir_base + raw)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. sample_submission.csv
  02. test.csv
  03. train.csv


In [17]:
submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 


submission['winner'] = pred
submission.to_csv(dir_base + submit + filename_submit, index=False)

# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [18]:
df_1 = submission
df_1

,game_id,winner
0,38872,0.586648
1,38873,0.488454
2,38874,0.456282
3,38875,0.372868
4,38876,0.462080
...,...,...
16782,55654,0.548557
16783,55655,0.427770
16784,55656,0.719651
16785,55657,0.527545


In [19]:
%%time
"""
# df_time 데이터를 만든다. = 유니크 게임 갯수 *** DATA SHAPE = [ 38,872 x 1 ]
"""

df_time = pd.DataFrame(columns=['game_id'])       # 빈 df 선언
df_time['game_id'] = test['game_id'].unique() # 빈 df 에 'game_id 를 채운다
df_time['time'] = np.array(test[test.shift(-1).game_id != test.game_id].time)

CPU times: user 6.84 s, sys: 1.38 s, total: 8.22 s
Wall time: 2 s


In [20]:
df_time

,game_id,time
0,38872,7.06
1,38873,6.07
2,38874,4.41
3,38875,7.58
4,38876,2.50
...,...,...
16782,55654,6.14
16783,55655,9.58
16784,55656,6.24
16785,55657,1.17
